Electric Propulsion Test Code #1

In [2]:
import matplotlib.pyplot as plt
from math import sqrt
import pandas as pd
import numpy as np

Inputs

In [4]:
data = pd.read_excel('Electric_Rockets_and_Solar_Panels.xlsx')
# based on the excel choose the row location of the one you want
iont = 3
hallt = 0
solarp = 6

In [5]:
# From Solar Panels
pp = data.iloc[solarp,1] # power per panel
spweight = data.iloc[solarp,2] # solar panel weight
paneleff = data.iloc[solarp,3] # this is in kW/m^3

In [15]:
# From Ion Thruster Chosen
ionisp = data.iloc[iont,1]
ionthrust = data.iloc[iont,2]
ioneff = data.iloc[iont,3]
ionfuelmw = data.iloc[iont,4]
ionweight = data.iloc[iont,5]
# From Hall Thruster Chosen
hallisp = data.iloc[hallt,1]
hallthrust = data.iloc[hallt,2]
halleff = data.iloc[hallt,3]
hallfuelmw = data.iloc[hallt,4]
hallweight = data.iloc[hallt,5]

Electric Prop Equations

In [17]:
ge = 9.81 # earth gravity
a = 7.34 # q/m for xenon got it from the book (10^5 C/kg)
# Ion Thrusters
ionVa = ((ionisp*ge)**2)/(2*(ge/ionfuelmw))
ionEP = (ionthrust*ionisp*ge)/(2*ioneff)
ionI = ionEP/ionVa
ionUe = (2*a*ionVa)/ionfuelmw
ionmdot = (2*ionEP)/(ionUe**2)
# Hall Thrusters
hallVa = ((hallisp*ge)**2)/(2*(ge/hallfuelmw))
hallEP = (hallthrust*hallisp*ge)/(2*halleff)
hallI = hallEP/hallVa
hallUe = (2*a*hallVa)/hallfuelmw
halldot = (2*hallEP)/(hallUe**2)